In [38]:
import os
import json
import glob
import numpy as np
from pathlib import Path
import duckdb
import pandas as pd
import pyarrow as pa

import tqdm

from natsort import natsorted, index_natsorted


from gensim.models import Word2Vec, FastText 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

In [35]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/oceansxyz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/oceansxyz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
df = pd.read_csv("pali_canon.csv", index_col='index')

In [23]:
df_processed = df[df["sutta_id"] == "mn1"]["text"].copy()
df_processed = df_processed.str.lower()

if "mn1:0.1" in df_processed.index:
    df_processed = df_processed.drop(index='mn1:0.1')




df_processed.head()

index
mn1:0.2                                   mūlapariyāyasutta 
mn1:1.1                                       evaṁ me sutaṁ—
mn1:1.2    ekaṁ samayaṁ bhagavā ukkaṭṭhāyaṁ viharati subh...
mn1:1.3                 tatra kho bhagavā bhikkhū āmantesi: 
mn1:1.4                                      “bhikkhavo”ti. 
Name: text, dtype: str

In [44]:
chunks = list(df_processed)

chunks[13]

'pathaviṁ pathavito saññatvā pathaviṁ maññati, pathaviyā maññati, pathavito maññati, pathaviṁ meti maññati, pathaviṁ abhinandati. '

In [46]:
word_tokens = [word_tokenize(chunk.lower()) for chunk in chunks]

In [50]:
model = Word2Vec(
    sentences=word_tokens,
    vector_size=100,   # small for testing
    window=5,          # context window size
    min_count=1,       # include all words
    workers=4,         # CPU threads
    sg=1               # skip-gram model
)

In [52]:
word = "bhagavā"
if word in model.wv:
    print(f"Vector for '{word}':\n", model.wv[word][:10], "...")  # first 10 dims
    # Find similar words
    print("Most similar words:", model.wv.most_similar(word, topn=5))
else:
    print(f"'{word}' not in vocabulary")

Vector for 'bhagavā':
 [-0.00650317  0.00127647 -0.00190314  0.00905263  0.00979077 -0.0243803
  0.01041334  0.04735521 -0.02808947 -0.02143994] ...
Most similar words: [('bhikkhave', 0.9459238648414612), ('bhikkhu', 0.9452006816864014), (',', 0.9450480341911316), ('nibbānato', 0.9446079134941101), ('maññati', 0.9443570971488953)]


In [ ]:
sent_tokenize(chunks[13])

IndexError: list index out of range